# libs

In [1274]:
import copy
import midi
import numpy as np
import pandas as pd
import random

# https://code.google.com/archive/p/python-musical/source/default/source
# pretty awesome library. I downloaded the package from the above link and then extracted
# the musical/ directory from python-musical/trunk/ and placed in sister midi-skirt/ (.gitignored). 
from musical.theory import Note, scale, Scale, Chord

# constants

In [1275]:
class PatternConstants:
    def __init__(self, resolution=440):
        self.resolution = resolution
        self.quarter_note = resolution
        self.half_note = resolution * 2
        self.eighth_note = int(resolution / 2.0)
        self.sixteenth_note = int(resolution / 4.0)
        self.thirty_second_note = int(resolution / 8.0)
        self.sixty_forth_note = int(resolution / 16.0)
        self.whole_note = resolution * 4
        self.bar = resolution * 4

pc = PatternConstants()

# pattern and track

In [1276]:
pattern = midi.Pattern(resolution=pc.resolution)
track = midi.Track()
#melody_track = midi.Track()
track.append(tempo)
pattern.append(track)
#pattern.append(melody_track)

# helper flunctions

In [1277]:
def make_ticks_rel(track):
    number_before_negative = 0
    running_tick = 0
    for event in track:
        event.tick -= running_tick
        if event.tick >= 0:
            number_before_negative += 1
        else:
            print number_before_negative
        running_tick += event.tick
    return track

In [1278]:
def get_max_tick(track):
    return max([event.tick for event in track])

def get_min_tick(track):
    return min([event.tick for event in track])

# event stager

In [1279]:
class MidiEventStager:
    # class to store events nearly ready to be converted to events
    def __init__(self, midi_event_fun, start_tick, duration, pitch, velocity):
        self.midi_event_fun = midi_event_fun
        self.start_tick = start_tick
        self.duration = duration
        self.pitch = pitch
        self.velocity = velocity
    
#     def apply_midi_event_fun():
#         self.midi_event_fun(tick=self.start_tick, velocity=self.velocity, pitch=self.pitch)
        
    def __repr__(self):
        return "{} {}".format(self.pitch, self.midi_event_fun.name)

In [1280]:
def convert_staging_events_to_dataframe(staging_events):
    event_tuples = [(se.midi_event_fun, se.start_tick, se.duration, se.pitch, se.velocity) for se in staging_events]
    event_df = pd.DataFrame(event_tuples)
    event_df.columns = ["event_type_fun", "tick", "duration", "pitch", "velocity"]
    return event_df

# chord

In [1281]:
class MidiChord:
    """Build a list of staging events from input notes.

    Note: a MidiChord has no conception of start_tick but set_start_tick helps set the tick.
    """
    # 
    def __init__(self, chord_notes=None):
        """
        chord_notes: a list of strings that represent the notes and octaves of the chord.
        usage:
            chord = MidiChord(['c1', 'e1', 'g1'])
        """
        self.chord_notes = chord_notes
        self.staged_events = []
        # self.chord_name = ""
    
    def build_chord(self):
        """Populates the staged_events list with on and off of events, one for each note in the chord."""
        for note in self.chord_notes:
            # velocity = random.randint(50, 90)
            notes_created = self._create_note_tuple(note)
            self.staged_events.extend(notes_created)
    
#     def set_start_tick(self, start_tick):
#         for event in self.staged_events:
#             event.start_tick += start_tick
    
#     def set_start_ticks(self, start_ticks):
#         raise NotImplemented

#     def set_start_tick_noisily(self, start_tick, noise_range):
#         for event in self.staged_events:
#             event.start_tick += start_tick + np.random.uniform(noise_range[0], noise_range[1])
            
    def _create_note_tuple(self, note):
        return [
            MidiEventStager(midi.NoteOnEvent, 0, 0, note, 0),
            MidiEventStager(midi.NoteOffEvent, 0, 0, note, 0)
        ]
        

In [1282]:
chord = MidiChord(['c1', 'e1', 'g1'])

In [1283]:
chord.build_chord()

In [1284]:
chord.staged_events

[c1 Note On, c1 Note Off, e1 Note On, e1 Note Off, g1 Note On, g1 Note Off]

# chord builder

In [1285]:
class ChordBuilder:
    """A class with functions to help create MidiChord instances."""

    def build_from_intervals(self, root, octave, intervals, scale_name="major"):
        """Given chord specs return a MidiChord object of said chord.
        
        root: string of the note.
        octave: an integer between 0 and 8 (or 9 or something)
        intervals: a list of note intervals relative to the root. Use 'b' for flat and '#' for sharp.
        
        usage:
            chord = ChordBuilder().build_from_intervals('c', 6, ["1", "3", "5", "b7", "#9"])
            
        returns:
            a MidiChord object
        """
        named_scale = scale.NAMED_SCALES[scale_name]
        my_scale = Scale(Note((root.upper(), octave)), named_scale)
        num_notes_in_scale = len(my_scale)
        scale_start_num = octave * num_notes_in_scale
        intervals = [self._deal_with_pitch_accidentals(interval) for interval in intervals]
        notes = [my_scale.get(scale_start_num+interval[0]-1).transpose(interval[1]) for interval in intervals]
        chord = MidiChord([note.note + str(note.octave) for note in notes])
        chord.build_chord()
        return chord
    
    def build_directly(self):
        pass

    def build_randomly_from_scale():
        pass
    
    def _get_list_of_chord_notes_from_chord(self, notes):
        return [Note.index_from_string(chord_note.note + str(chord_note.octave)) for chord_note in notes]
    
    def _deal_with_pitch_accidentals(self, interval):
        # input "b9" output (9, -1)
        # input "#9" output (9, 1)
        # input "9" output (9, 0)
        if "b" in interval:
            transposition = -1
        elif "#" in interval:
            transposition = 1
        else:
            transposition = 0
        note = int(interval.replace("b", "").replace("#", ""))
        return((note, transposition))

In [1286]:
chord = ChordBuilder().build_from_intervals("Bb", 6, ["1", "3", "5", "b7", "#9"])

In [1287]:
notes = chord.staged_events

In [1288]:
notes

[a#6 Note On,
 a#6 Note Off,
 d7 Note On,
 d7 Note Off,
 f7 Note On,
 f7 Note Off,
 g#7 Note On,
 g#7 Note Off,
 c#8 Note On,
 c#8 Note Off]

# chord progression

In [1289]:
class ChordProgression:
    """A ChordProgression consists of chords and changes (start ticks)
    
    The changes are just the underlying chord progression. A rhythm is created separately and applied
    later by using the chord progression object to know which chord to play.
    """
    def __init__(self, chords=[], changes=[]):
        self.chords = chords
        self.changes = changes
        #self.changes.append(np.inf).insert(0, 0)

#     def build_progression_from_chords(self, chords):
#         """chords is a list of MidiChord objects"""
#         self.chords = chords
    
    def build_progression_randomly_from_scale(self, scale):
        pass
    
    def build_changes_randomly(self, duration_choices):
        pass
    
#     def build_changes_directly(self, changes):
#         """changes are how long to play each chord."""
#         self.changes = changes
    
    def repeat_progression(self, num_repeats):
        temp_chords = []
        temp_changes = []
        for _ in range(num_repeats):
            for chord in self.chords:
                temp_chords.append(copy.deepcopy(chord))
            for change in self.changes:
                temp_changes.append(copy.deepcopy(change))
        self.chords = temp_chords
        self.changes = temp_changes
        
    @property
    def length(self):
        return sum(self.changes)
    
    def get_chord_from_tick(self, tick):
        temp = [0] + self.changes + [np.inf]
        pos = np.sum(np.cumsum(temp) < tick)#  - 1
        if pos == len(self.chords):
            pos -= 1  # hack for now
        return self.chords[pos]
    
#     def build_changes_from_pattern(pattern, progression_length):
#         self.changes = np.cumsum(np.repeat(pattern, np.floor(float(progression_length) / sum(pattern))))
        

In [1290]:
c7sharp9 = ChordBuilder().build_from_intervals("C", 5, ["1", "3", "5", "b7", "#9"])
am7 = ChordBuilder().build_from_intervals("A", 5, ["1", "b3", "5", "b7"])
g6 = ChordBuilder().build_from_intervals("G", 5, ["1", "3", "5", "6"])
f69 = ChordBuilder().build_from_intervals("F", 5, ["1", "3", "5", "6", "9"])
bb13 = ChordBuilder().build_from_intervals("Bb", 5, ["1", "3", "5", "b7", "9", "13"])

In [1291]:
chord_progression = ChordProgression(
    chords=[am7, g6, f69, bb13],
    changes=[pc.bar, pc.bar, pc.bar, pc.bar]
)

In [1292]:
chord_progression.repeat_progression(18)

# rhythm

In [1293]:
class Rhythm:
    def __init__(self, rhythm_len=None, start_tick=None, quantization=None):
        self.rhythm_len = rhythm_len
        self.start_tick = start_tick
        self.quantization = quantization
        self.start_ticks = []
        self.note_lengths = []
    
    def build_rhythm_randomly(self, note_density, note_len_choices):
        number_of_notes = self._compute_num_notes(note_density)
        self.start_ticks = self._get_random_start_ticks(number_of_notes)
        self.note_lengths = [np.random.choice(note_len_choices) for _ in self.start_ticks]
    
    def build_rhythm_directly(self):
        pass
    
    def _compute_num_notes(self, note_density):
        return int(self.rhythm_len * note_density / float(self.quantization))
    
    def _get_random_start_ticks(self, number_of_notes):
        return np.unique(sorted([
                    self._find_nearest_note(
                        value=random.randint(0, self.rhythm_len)+self.start_tick,
                        note_type=self.quantization)
                    for _ in range(number_of_notes)]))
    
    def _find_nearest_note(self, value, note_type):
        mod = value % note_type
        if (mod > (note_type / 2.0)): # round up 
            return value + note_type - mod
        else:
            return value - mod


In [1294]:
rhythm = Rhythm(rhythm_len=pc.bar*64, start_tick=2, quantization=pc.sixteenth_note)
# rhythm = Rhythm(rhythm_len=pc.bar*64, start_tick=2, quantization=pc.bar)

In [ ]:
rhythm.build_rhythm_randomly(
    note_density=.5,
    note_len_choices=[pc.sixteenth_note, pc.thirty_second_note, pc.eighth_note, pc.quarter_note, pc.whole_note])

In [1295]:
# rhythm.build_rhythm_randomly(
#     note_density=10.0,
#     note_len_choices=[pc.whole_note])

# chord progression + rhythm

In [1296]:
class ChordProgressionRhythm:
    def __init__(self, rhythm, chord_progression, match_lengths=True):
        self.rhythm = rhythm
        self.chord_progression = chord_progression
        self.chords = self.build_staging_events()
        if match_lengths:
            self.make_chord_and_rhythm_same_length()
    
    def build_staging_events(self):
        chords = []
        for tick, duration in zip(self.rhythm.start_ticks, self.rhythm.note_lengths):
            chord = copy.deepcopy(self.chord_progression.get_chord_from_tick(tick))
            for event in chord.staged_events:
                if event.midi_event_fun.name == "Note On":
                    event.start_tick = tick
                    event.duration = duration
                    event.velocity = random.randint(50, 90)
                elif event.midi_event_fun.name == "Note Off":
                    event.start_tick = tick + duration
                    event.duration = duration
                else:
                    print 'wat'
            chords.append(chord)
        print chords[10].staged_events[0].start_tick
        return chords

    def make_chord_and_rhythm_same_length(self):
        pass

In [1297]:
cpr = ChordProgressionRhythm(rhythm, chord_progression)

17600


In [1298]:
all_staged_events = []
for chord in cpr.chords:
    for staged_event in chord.staged_events:
        # print staged_event.start_tick
        all_staged_events.append(staged_event)

In [1299]:
df = convert_staging_events_to_dataframe(all_staged_events)

In [1300]:
df.sort_values(by=["tick", "duration"], inplace=True)

In [1301]:
df.head(5)

,event_type_fun,tick,duration,pitch,velocity
0,<class 'midi.events.NoteOnEvent'>,0,1760,a5,58
2,<class 'midi.events.NoteOnEvent'>,0,1760,c6,83
4,<class 'midi.events.NoteOnEvent'>,0,1760,e6,76
6,<class 'midi.events.NoteOnEvent'>,0,1760,g6,89
1,<class 'midi.events.NoteOffEvent'>,1760,1760,a5,0


In [1302]:
def add_tuples_to_track(track, df):
    track.append(midi.InstrumentNameEvent(tick=0, text='Classic Electric Piano', data=[]))
    for row in df.iterrows():
        data = row[1]
        track.append(data["event_type_fun"](tick=data["tick"],
                                            velocity=data["velocity"],
                                            pitch=Note.index_from_string(data["pitch"])))
    return track

In [1303]:
track = add_tuples_to_track(track, df)

# melody

In [1304]:
# class Melody:
#     def __init__(self, melody_len=None, scale=None, quantization=None, note_density=None, note_len_choices=None,
#                  available_notes=None):
#         self.melody_len = melody_len
#         self.quantization = quantization  # this maybe should be at note level only
#         self.note_density = note_density  # proportion of available ticks (determined by quantization) occupied by notes
#         self.note_len_choices = note_len_choices
#         self.available_notes = available_notes
#         self.number_of_notes = self._compute_num_notes()
#         self.start_ticks = self._get_start_ticks()
    
#     def _compute_num_notes(self):
#         return self.melody_len / self.quantization
    
#     def _get_start_ticks(self):
#         return np.unique(sorted([find_nearest_note(random.randint(0, self.melody_len), self.quantization)
#                                  for _ in range(self.number_of_notes)]))

#     def create_melody(self):
#         melody_notes = []
#         for tick in self.start_ticks:
#             melody_tuples = self._create_melody_note_tuple(tick)
#             melody_notes.extend(melody_tuples)
#         return melody_notes
    
#     def _create_melody_note_tuple(self, start_tick):
#         note_id = str(uuid.uuid1())
#         velocity = random.randint(50, 90)
#         cur_note = random.choice(self.available_notes)
#         cur_note = Note.index_from_string(cur_note.note + str(cur_note.octave))
#         note_length = random.choice(self.note_len_choices)
#         return [
#             (note_id, midi.NoteOnEvent, start_tick, cur_note, note_length, velocity),
#             (note_id, midi.NoteOffEvent, start_tick+note_length, cur_note, note_length, 0)
#         ]

In [1305]:
# # note2self: hard-coded in melody: velocity of notes, octave
# melody = Melody(
#     melody_len=pc.bar*64,
#     quantization=pc.sixteenth_note,
#     note_density=.4,
#     note_len_choices=[pc.quarter_note, pc.eighth_note, pc.sixteenth_note, pc.sixty_forth_note,
#                       pc.half_note, pc.thirty_second_note],
#     available_notes=[my_scale.get(x) for x in range(28, 37)])

# my_melody = melody.create_melody()

In [1306]:
# rhythm_track = add_tuples_to_track(rhythm_track, my_rhythm)
# melody_track = add_tuples_to_track(melody_track, my_melody)

# end of track

In [1307]:
# Add the end of track event, append it to the track
eot = midi.EndOfTrackEvent(tick=get_max_tick(track) + 2*pc.whole_note)  # probably an excessive buffer
track.append(eot)

# eot = midi.EndOfTrackEvent(tick=get_max_tick(melody_track) + 2*pc.whole_note)
# melody_track.append(eot)

In [1308]:
track = make_ticks_rel(track)
# melody_track = make_ticks_rel(melody_track)

In [1309]:
# track

# write to file

In [1310]:
midi.write_midifile("example.mid", pattern)

In [1311]:
import os

In [1312]:
# os.system("timidity /Users/jacknorman1/Documents/Programming/midi-skirt/example.mid")